In [1]:
from CONFIG import *

In [2]:
import torch
import torch.nn as nn

In [3]:
from Our_VGG16 import vgg

In [4]:
net = vgg()

In [5]:
net.classifier = nn.Sequential(nn.Linear(512*7*7,4096),nn.ReLU(True),nn.Dropout(),nn.Linear(4096,1470))

In [6]:
net.load_state_dict(torch.load('best.pth'))

In [33]:
net.cuda()

vgg(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    

In [7]:
import torchvision.transforms as transforms

In [8]:
from torch.autograd import Variable

In [11]:
import os

In [14]:
import cv2

In [18]:
import numpy as np

In [72]:
def predictor(net,img_name,root_path = ''):
    img = cv2.imread(root_path+img_name)
    height,width,channels = img.shape
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    mean = np.mean(img,axis = (0,1))
    img = img - mean
    img = cv2.resize(img,(224,224))
    t = transforms.Compose([transforms.ToTensor()])
    img = t(img)
    print(img.shape)
    img = Variable(img[None,:,:,:], volatile = True) #Volatile = True is not done here, try it if things aren't working
    print(img.shape)
    img = img.cuda()
    prediction = net(img)
    prediction.cpu()
    predictions = []
    prediction = prediction.data.squeeze(0)
    bb1_confidence = prediction[:,:,4].unsqueeze(2)
    bb2_confidence = prediction[:,:,9].unsqueeze(2)
    both_confidences = torch.cat((bb1_confidence,bb2_confidence),2)
    if both_confidences.max() < confidence_threshold:
        bb_in_cell = both_confidences==both_confidences.max()
    else:
        bb_in_cell = both_confidences>confidence_threshold
    class_probabilities = []
    bounding_boxes = []
    class_indexes = []
    bbox_confidence = []
    for i in range(cells_per_row):
        for j in range(cells_per_row):
            cell_data = prediction[i,j,:]
#             print "cell_data type : ",type(cell_data)
            if bb_in_cell[i,j,0] == 1:
                bbox_1 = cell_data[:4]
                bbox_1.cpu()
                bbox_1_confidence = torch.FloatTensor([cell_data[4]])
                top_left = torch.FloatTensor([j,i])*cell_size
#                 bbox_1[:2] = bbox_1[:2]*cell_size + top_left
                print("(bbox_1[:2])",type(bbox_1[:2]),len(bbox_1[:2]))
                print("(bbox_1[:2]*cell_size)",type(bbox_1[:2]*cell_size),len(bbox_1[:2]*cell_size))
                print("top_left :",type(top_left),len(top_left))
                print(bbox_1[:2])
                bbox_1[:2] = bbox_1[:2]*cell_size + top_left
                bbox_1_xy = torch.FloatTensor(bbox_1.size())
                bbox_1_xy[:2] = bbox_1[:2] - (bbox_1[2:]/2)
                bbox_1_xy[2:] = bbox_1[:2] + (bbox_1[2:]/2)
                print("len(cell_data[10:]) :",len(cell_data[10:]))
                highest_prob,class_index = torch.max(cell_data[10:],0)
                bounding_boxes.append(bbox_1_xy.view(1,4))
                class_indexes.append(class_index)
                class_probabilities.append(highest_prob)
                bbox_confidence.append(bbox_1_confidence)
            elif bb_in_cell[i,j,1] == 1:
                bbox_2 = cell_data[5:9]
                bbox_2_confidence = torch.FloatTensor([cell_data[9]])
                top_left = torch.FloatTensor([j,i])*cell_size
                bbox_2[:2] = bbox_2[:2]*cell_size + top_left
                bbox_2_xy = torch.FloatTensor(bbox_2.size())
                bbox_2_xy[:2] = bbox_2[:2] - (bbox_2[2:]/2)
                bbox_2_xy[2:] = bbox_2[:2] + (bbox_2[2:]/2)
                highest_prob,class_index = torch.max(cell_data[10:],0)
                bounding_boxes.append(bbox_2_xy.view(1,4))
                class_indexes.append(class_index)
                class_probabilities.append(highest_prob)
                bbox_confidence.append(bbox_2_confidence)
    bounding_boxes = torch.cat(bounding_boxes,0)
    bbox_confidence = torch.cat(bbox_confidence,0)
    class_indexes = torch.cat(class_indexes,0)
    
    for i,box in enumerate(bounding_boxes):
        x1 = int(box[0]*width)
        y1 = int(box[1]*height)
        x2 = int(box[2]*width)
        y2 = int(box[3]*height)
        c = int(class_indexes[i])
        p = float(bbox_confidence[i])
        predictions.append([(x1,y1),(x2,y2),VOC_CLASSES[c],image_name,p])
    
    return predictions
        
                
                
            
    
    
        
    

In [73]:
root_dir = 'VOC_Datasets/VOC_2012_training/VOC2012/JPEGImages/'
img_names = os.listdir(root_dir)
for i,name in enumerate(img_names):
    if i < 15:
        pass
    else:
        image_name = root_dir+name
        print(image_name)
        image = cv2.imread(image_name)
        result = predictor(net,image_name)
        for left_up,right_bottom,class_name,_,prob in result:
            cv2.rectangle(image,left_up,right_bottom,(0,255,0),2)
            cv2.putText(image,class_name,left_up,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,2)
            print(prob)

        cv2.imwrite('train_result'+str(i)+'.jpg',image)
    if i > 30:
        break

VOC_Datasets/VOC_2012_training/VOC2012/JPEGImages/2008_008683.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
('(bbox_1[:2])', <class 'torch.cuda.FloatTensor'>, 2)
('(bbox_1[:2]*cell_size)', <class 'torch.cuda.FloatTensor'>, 2)
('top_left :', <class 'torch.FloatTensor'>, 2)

 0.7622
 0.7615
[torch.cuda.FloatTensor of size 2 (GPU 0)]



TypeError: add received an invalid combination of arguments - got (torch.FloatTensor), but expected one of:
 * (float value)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.FloatTensor[0m)
 * (torch.cuda.FloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.FloatTensor[0m)
 * (torch.cuda.sparse.FloatTensor other)
      didn't match because some of the arguments have invalid types: ([31;1mtorch.FloatTensor[0m)
 * (float value, torch.cuda.FloatTensor other)
 * (float value, torch.cuda.sparse.FloatTensor other)


In [8]:
import cv2

In [9]:
net.eval()
if torch.cuda.is_available():
    net.cuda()

    
test_img = cv2.imread('Cat.jpg')

In [14]:
test_img = cv2.imread('Cat.jpg')
test_img.shape

(177, 284, 3)

In [12]:
import numpy as np

In [16]:
test_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2RGB)

In [17]:
np.mean(test_img,axis =(0,1))

array([191.53859314, 177.46560436, 170.1299435 ])

In [58]:
torch.max?